In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
from langchain.chains import ConversationChain
from langchain.memory import  ConversationBufferWindowMemory
from langchain_core.prompts.prompt import PromptTemplate

In [10]:
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

In [4]:
# setting up groq api key
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

# chat set up
GROQ_LLM = ChatGroq(temperature=0.5, model_name="llama3-8b-8192")

## Basic Chains

- 1. Chat Chain
- 2. Initiator
- 3. Chat Reviewer
- 4. Requirement Extractor
- 5. Problem Search
- 6. Code Generation
- 7. Code and Requirement Reviewer

## Other Functions
- 1. Code Saver
- 2. Class Generator
- 3. Code implementation and data saver.  

### Chat Chain

In [8]:
# Now we can override it and set it to "AI Assistant"

chat_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a Data Science Consultant. You have over 20 years of experience in the field. 
You are currently working with a client to create a synthetic data for his project. 
The data is a real world based daa exhibits the same characteristics as the real data.

Initially you don't know anything about the project, that's why you want to ask the client some questions to understand the data requirements.
This is the work flow you have been following:
    1. Initiate a besutiful and interactive sessons with the user.
    2. Ask questions about the project the client is working on.
    3. Ask for the purpose of this data in relation to the project.
    4. Ask for possible columns and the data types. You can also suggest based on the project.
    5. After for the intended data size. You can also suggest based on the project. The client can also specify the size.
    6. Collate all the information and present back to the client for review.
    7. Review thw generated code with the client requirements.
    8. After User has verified that all the requirements are met, respond with - ALL REQUIREMENTS RECORDED.

<|eot_id|><|start_header_id|>user<|end_header_id|>
Current conversation:
{history}
Human: {input}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
AI Assistant:"""

PROMPT = PromptTemplate(input_variables=["history", "input"], template=chat_template)
chat_conversation = ConversationChain(
    prompt=PROMPT,
    llm=GROQ_LLM,
    verbose=True,
    memory=ConversationBufferWindowMemory(k=10, ai_prefix="AI Assistant"),
    output_parser=StrOutputParser(),
)

In [9]:
chat_conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a Data Science Consultant. You have over 20 years of experience in the field. 
You are currently working with a client to create a synthetic data for his project. 
The data is a real world based daa exhibits the same characteristics as the real data.

Initially you don't know anything about the project, that's why you want to ask the client some questions to understand the data requirements.
This is the work flow you have been following:
    1. Initiate a besutiful and interactive sessons with the user.
    2. Ask questions about the project the client is working on.
    3. Ask for the purpose of this data in relation to the project.
    4. Ask for possible columns and the data types. You can also suggest based on the project.
    5. After for the intended data size. You can also suggest based on the project. The client can also specify the size.
    

"Hello! It's great to meet you! I'm your Data Science Consultant, and I'll be helping you with creating synthetic data for your project. I'd love to learn more about your project and understand your requirements.\n\nCan you please tell me a bit about the project you're working on? What's the main goal or objective of your project?"

### Initiator

In [193]:
def initiator(chat_history: list) -> str:
    initiator_prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are an expert at evaluating the conversation between a USER and an AI. \n
        Your focus is on the AI response. You're looking for a place where the AI response has "ALL REQUIREMENTS RECORDED" in it. \n
        
        If you see "ALL REQUIREMENTS RECORDED" in thre AI repsonse:
            Return 'START' as your response. \
        Else if it's not there:
            Return 'WAIT' as your response
        
        Do not make assumptions. Stick to the details in the chat. 
        You can only return "START" or "WAIT", no other explanation is needed.
        <|eot_id|><|start_header_id|>user<|end_header_id|>
        CHAT_HISTORY: {chat_history} \n
        <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["chat_history"],
    )

    initiator_router = initiator_prompt | GROQ_LLM | StrOutputParser()
    output = initiator_router.invoke({"chat_history":chat_history})
    return output

### Chat Reviewer

In [123]:
## Rewrite Router
def chat_reviewer(chat_history: list) -> str:
    chat_reviewer = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are an expert at evaluating the conversation between a USER and an AI. \n
        You are checking to see if the AI has already understood the user requirements and if the user approves of it too. \n

        Ask yourself the following questions: \n\n

        Has the user mentioned all of his requirements?
        Has the AI understood the requirements correctly?
        Has the AI presented the requirements back to the user for verification?
        Has the user approved of the requirements?

        Give a binary choice 'understood' (if the requirements has been met)) or 'not_understood' (if the ai still needs more information) based on the chat conversation.
        Return only the binary choice and no premable or explaination. \
        Be Sure of your decision before you submit. \n
        <|eot_id|><|start_header_id|>user<|end_header_id|>
        CHAT_HISTORY: {chat_history} \n
        <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["chat_history"],
    )

    chat_review_router = chat_reviewer | GROQ_LLM | StrOutputParser()
    output = chat_review_router.invoke({"chat_history":chat_history})
    return output


In [124]:
CHAT_HISTORY = [{
    "messages": [
        {"user": "USER", "text": "Hi, how are you today?", "timestamp": "2023-02-02T12:00:00"},
        {"ai": "AI", "text": "Hello! I'm doing well, thank you for asking. How can I assist you?", "timestamp": "2023-02-02T12:00:05"},
        {"user": "USER", "text": "I need to book a flight from New York to Los Angeles on a specific date.", "timestamp": "2023-02-02T12:10:00"},
        {"ai": "AI", "text": "I'd be happy to help you with that! What is the specific date you're looking to travel?", "timestamp": "2023-02-02T12:11:00"},
        {"user": "USER", "text": "February 14th.", "timestamp": "2023-02-02T12:12:00"},
        {"ai": "AI", "text": "Let me check availability. Can you please confirm that you'd like to travel on February 14th from New York to Los Angeles?", "timestamp": "2023-02-02T12:13:00"},
        {"user": "USER", "text": "Yes, that's correct.", "timestamp": "2023-02-02T12:14:00"},
        {"ai": "AI", "text": "I've found a suitable flight for you. Would you like to book it?", "timestamp": "2023-02-02T12:15:00"}
    ]
}]

In [125]:
print(chat_reviewer(CHAT_HISTORY))

understood


### Requirement Extraction

In [126]:
## Requirement Extractor
def requirement_extractor(chat_history: list, chat_review: str) -> dict:
    requirement_extractor_prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are the Coding Requirement Extraction Agent take the CHAT_HISTORY of a conversation between the USER and AI, the CHAT_REVIEW  \
                which tells if the ai understands the user requirment \
                If the CHAT_REVIEW is 'understood' then extract the code requirement from the chat_history. \n
                
                Extract a code requirement in this format:
                    1. Problem statement: The problem that the user is trying to solve.
                    2. Column names: The columns that the user wants in the data.
                    3. Data types: The data types for each column.
                    4. Data size: The intended size of the data. This should be the number of rows.
                    5. Data Variability: The possible variability in the data as per the user's requirement. This is optional.

                
                You never make up information that hasn't been provided by the CHAT_HISTORY. \n
                Return the code requirement as a JSON with the keys 'problem_statement', 'column_names', 'data_types', 'data_size', and 'data_variability' if provided. \n
                If the CHAT_REVIEW is 'not_understood' then return 'Nothing-To-Extract' as JSON with a single key 'no_extraction' and no premable or explaination. \n

                
        <|eot_id|><|start_header_id|>user<|end_header_id|>
        CHAT_HISTORY: {chat_history} \n
        CHAT_REVIEW: {chat_review} \n
        <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["chat_history","chat_review"],
    )

    requirement_extraction_chain = requirement_extractor_prompt | GROQ_LLM | JsonOutputParser()
    output = requirement_extraction_chain.invoke({"chat_history":chat_history, "chat_review":chat_review})
    return output

In [128]:
CHAT_HISTORY =  {
    "messages": [
        {"user": "USER", "text": "Hi, I'm trying to create a dataset for sales in a city. I'm looking to analyze the sales of fashion products in New York City. I want to track the sales of specific products to understand their market performance.", "timestamp": "2023-02-10T14:25:00"},
        {"ai": "AI", "text": "Sounds like an interesting project! Can you please tell me more about what you're trying to achieve? What specific problem are you trying to solve?", "timestamp": "2023-02-10T14:26:00"},
        {"user": "USER", "text": "The problem I'm trying to solve is to analyze the sales trends of blouses, tops, and dresses in New York City. I want to understand which products are most popular and identify any seasonal patterns.", "timestamp": "2023-02-10T14:27:00"},
        {"ai": "AI", "text": "Got it! So, you're looking to analyze sales data for blouses, tops, and dresses in New York City. Can you please list the columns you'd like to include in the dataset?", "timestamp": "2023-02-10T14:28:00"},
        {"user": "USER", "text": "I'd like to include columns for date, product category (blouses, tops, dresses), product sub-category (e.g. long-sleeve blouses), and sales amount.", "timestamp": "2023-02-10T14:30:00"},
        {"ai": "AI", "text": "That makes sense! What data types do you expect for each column? For example, would the sales amount be a numerical value?", "timestamp": "2023-02-10T14:31:00"},
        {"user": "USER", "text": "Yes, the sales amount would be a numerical value. And the date would be a date type. The product categories would be categorical.", "timestamp": "2023-02-10T14:33:00"},
        {"ai": "AI", "text": "Got it! For the sales amount, I assume you want a numerical value, such as a float or integer. For the date, I assume it would be a date data type. And for the product categories, I assume it would be a categorical value, such as strings or categorical data types like enum or categorical.", "timestamp": "2023-02-10T14:34:00"},
        {"user": "USER", "text": "Yes, that's correct! I'd also like to know how many rows of data you'd like to generate.", "timestamp": "2023-02-10T14:35:00"},
        {"ai": "AI", "text": "I can generate a dataset with a specified number of rows. How many rows would you like the dataset to have?", "timestamp": "2023-02-10T14:36:00"},
        {"user": "USER", "text": "I'd like the dataset to have 10,000 rows.", "timestamp": "2023-02-10T14:37:00"},
        {"ai": "AI", "text": "Okay, I'll generate a dataset with 10,000 rows. Would you like to specify any variability in the data?", "timestamp": "2023-02-10T14:38:00"},
        {"user": "USER", "text": "No, I don't need any variability in the data.", "timestamp": "2023-02-10T14:39:00"},
        {"ai": "AI", "text": "Alright, I've generated the dataset with the specified columns, data types, and number of rows. I've excluded variability to ensure consistent data.", "timestamp": "2023-02-10T14:40:00"}
    ]
}


In [129]:
# chat_review_ = chat_review_router.invoke({"chat_history": CHAT_HISTORY})

# print(chat_review_)

In [130]:
print(requirement_extractor(CHAT_HISTORY, "understood"))

{'problem_statement': "The problem I'm trying to solve is to analyze the sales trends of blouses, tops, and dresses in New York City. I want to understand which products are most popular and identify any seasonal patterns.", 'column_names': ['date', 'product_category', 'product_sub_category', 'sales_amount'], 'data_types': {'date': 'date', 'product_category': 'categorical', 'product_sub_category': 'categorical', 'sales_amount': 'numerical'}, 'data_size': 10000, 'data_variability': 'none'}


### Code Generator

In [183]:
## Draft Code
def code_generator(chat_history: list, chat_review: str, extracted_requirements: dict) -> str:
    code_generator_prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are the Data Science Coding Agent. You read the CHAT_HISTORY below, the conversation between the USER and AI. \
        , the CHAT_REVIEW that reviews the chat and tells if the AI understands the USER requirements, \
        the Extracted Requirements from REQUIREMENT_EXTRACTOR.

        Here is your task:

            1. You write excellent code for the client based on the Extracted Requirements.
            2. Analyze the Extracted Requirements and make sure the code meets the requirements.
            3. You can use Pandas, Numpy, or any other library you think is necessary.
            4. You are to always ensure that the code is well-documented and easy to understand.
            5. Your priority is to write clean, efficient, and effective code that meets the client's requirements.
            6. Always check the data types compatibility to avoid errors. Use the libraries effectively to generate the data.
            7. You are to return the code as a string and no preamble or explanation.

            
        Other Informations: \n
            You never make up information that hasn't been provided by the CHAT_HISTORY, CHAT_REVIEW, or REQUIREMENT_EXTRACTOR. \n
            The CHAT_REVIEW is either 'understood' or 'not_understood'. If CHAT_REVIEW is 'understood', \
            the Extracted Requirements will be in the format of a JSON with the keys 'problem_statement', 'column_names', 'data_types', 'data_size', and 'data_variability' if provided. \

            If the CHAT_REVIEW is 'not_understood' then the Extracted Requirements will be 'Nothing-To-Extract' or 'True' as JSON with a single key 'no_extraction'. \
            So when we have 'Nothing-To-Extract' then we don't have any requirements to work with. \
            Else you return 'no-code generated' as a string and no premable or explanation. \


        <|eot_id|><|start_header_id|>user<|end_header_id|>
        CHAT_HISTORY: {chat_history} \n\n
        CHAT_REVIEW: {chat_review} \n\n
        REQUIREMENT_EXTRACTOR: {extracted_requirements} \n\n

        <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["chat_history","chat_review","extracted_requirements"],
    )

    code_generator_chain = code_generator_prompt | GROQ_LLM | StrOutputParser()
    output = code_generator_chain.invoke({"chat_history":chat_history, "chat_review":chat_review, "extracted_requirements":extracted_requirements})
    return output

In [184]:
chat_review_ =  chat_reviewer(CHAT_HISTORY)
extracted_req = requirement_extractor(CHAT_HISTORY, chat_review_)
print(chat_review_)
print(extracted_req)

understood
{'problem_statement': 'Analyze the sales trends of blouses, tops, and dresses in New York City to understand which products are most popular and identify any seasonal patterns.', 'column_names': ['date', 'product_category', 'product_sub_category', 'sales_amount'], 'data_types': {'date': 'date', 'product_category': 'categorical', 'product_sub_category': 'categorical', 'sales_amount': 'float'}, 'data_size': 10000, 'data_variability': 'none'}


In [185]:
code_output = code_generator(CHAT_HISTORY, chat_review_, extracted_req)

In [186]:
print(code_output)

Here is the code based on the extracted requirements:

```
import pandas as pd
import numpy as np

# Define the column names and data types
column_names = ['date', 'product_category', 'product_sub_category', 'sales_amount']
data_types = {'date': 'datetime64[ns]', 
             'product_category': 'category', 
             'product_sub_category': 'category', 
             'sales_amount': 'float64'}

# Generate the dataset with 10,000 rows
data = {'date': pd.date_range(start='2023-01-01', periods=10000), 
        'product_category': np.random.choice(['blouses', 'tops', 'dresses'], size=10000), 
        'product_sub_category': np.random.choice(['long-sleeve blouses', 'short-sleeve blouses', 'tunics', 't-shirts', 'dresses', 'rompers'], size=10000), 
        'sales_amount': np.random.uniform(0, 100, size=10000)}

# Convert the data to a pandas DataFrame
df = pd.DataFrame(data)

# Set the data types
df['date'] = pd.to_datetime(df['date'])
df['product_category'] = df['product_category'].astyp

In [187]:
exec('''
import pandas as pd
import numpy as np

# Define the column names and data types
column_names = ['date', 'product_category', 'product_sub_category', 'sales_amount']
data_types = {'date': 'datetime64[ns]', 
             'product_category': 'category', 
             'product_sub_category': 'category', 
             'sales_amount': 'float64'}

# Generate the dataset with 10,000 rows
data = {'date': pd.date_range(start='2023-01-01', periods=10000), 
        'product_category': np.random.choice(['blouses', 'tops', 'dresses'], size=10000), 
        'product_sub_category': np.random.choice(['long-sleeve blouses', 'short-sleeve blouses', 'tunics', 't-shirts', 'dresses', 'rompers'], size=10000), 
        'sales_amount': np.random.uniform(0, 100, size=10000)}

# Convert the data to a pandas DataFrame
df = pd.DataFrame(data)

# Set the data types
df['date'] = pd.to_datetime(df['date'])
df['product_category'] = df['product_category'].astype('category')
df['product_sub_category'] = df['product_sub_category'].astype('category')
df['sales_amount'] = df['sales_amount'].astype('float64')

''')

In [188]:
df

,date,product_category,product_sub_category,sales_amount
0,2023-01-01,tops,tunics,64.219674
1,2023-01-02,dresses,dresses,40.592222
2,2023-01-03,dresses,dresses,98.726931
3,2023-01-04,tops,tunics,40.172002
4,2023-01-05,tops,tunics,19.924830
...,...,...,...,...
9995,2050-05-14,blouses,long-sleeve blouses,11.438736
9996,2050-05-15,blouses,short-sleeve blouses,21.132757
9997,2050-05-16,dresses,short-sleeve blouses,2.121448
9998,2050-05-17,dresses,short-sleeve blouses,4.633319


### Code Rewriter

In [131]:
## Rewrite code
def code_rewriter(generated_code: str, extracted_requirements: dict) -> str:
    code_rewriter_prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are the Data Science Coding Agent. You read the GENERATED_CODE, and the Extracted Requirements from REQUIREMENT_EXTRACTOR. \n

        Here is your task: \n

            1. Extract the coding part in the GENERATED_CODE. \n
            2. You compare this with the Extracted Requirements, to ensure that the code meets the requirements. \n
            3. You are to check the code for any errors, inconsistencies, or missing requirements. \n
            4. You make a deep search and simulate the code to ensure that it works as expected. For every method used you check to make sure it is correct\n
            5. You are rewriting the code with a 100 percent correctness. \n
            6. You are to return the code as a string and no premable or explanation. \n
            7. You are to ONLY RETURN THE CORRECTED/REWRITTEN CODE. \n 


        Make sure you are 100 percent sure of your corrections before you submit. \n

        <|eot_id|><|start_header_id|>user<|end_header_id|>
        GENERATED_CODE: {generated_code} \n\n
        REQUIREMENT_EXTRACTOR: {extracted_requirements} \n\n

        <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["generated_code","extracted_requirements"],
    )

    code_rewriter_chain = code_rewriter_prompt | GROQ_LLM | StrOutputParser()
    output = code_rewriter_chain.invoke({"generated_code":generated_code, "extracted_requirements":extracted_requirements})
    return output

In [105]:
# rewritten_code = code_rewriter_chain.invoke({"generated_code": code_output, "extracted_requirements":extracted_req})

In [106]:
# print(rewritten_code)

```
import pandas as pd
import numpy as np
import datetime as dt

# Define the column names and their data types
column_names = ['date', 'product_category', 'product_sub_category', 'sales_amount']
data_types = {'date': 'datetime64[D]', 'product_category': 'category', 'product_sub_category': 'category', 'sales_amount': 'float64'}

# Define the data size
data_size = 10000

# Generate the dataset
data = {
    'date': [dt.datetime(2023, 1, 1) + dt.timedelta(days=i) for i in range(data_size)],
    'product_category': np.random.choice(['blouses', 'tops', 'dresses'], size=data_size),
    'product_sub_category': np.random.choice(['long-sleeve blouses', 'short-sleeve blouses', 'tunics', 't-shirts', 'dresses'], size=data_size),
    'sales_amount': np.random.randint(1, 100, size=data_size)
}

# Convert the data to a pandas DataFrame
df = pd.DataFrame(data)

# Print the generated dataset
print(df)
```


In [108]:
import pandas as pd
import numpy as np
import datetime as dt

# Define the column names and their data types
column_names = ['date', 'product_category', 'product_sub_category', 'sales_amount']
data_types = {'date': 'datetime64[D]', 'product_category': 'category', 'product_sub_category': 'category', 'sales_amount': 'float64'}

# Define the data size
data_size = 10000

# Generate the dataset
data = {
    'date': [dt.datetime(2023, 1, 1) + dt.timedelta(days=i) for i in range(data_size)],
    'product_category': np.random.choice(['blouses', 'tops', 'dresses'], size=data_size),
    'product_sub_category': np.random.choice(['long-sleeve blouses', 'short-sleeve blouses', 'tunics', 't-shirts', 'dresses'], size=data_size),
    'sales_amount': np.random.randint(1, 100, size=data_size)
}

# Convert the data to a pandas DataFrame
df = pd.DataFrame(data)

# Print the generated dataset
df

,date,product_category,product_sub_category,sales_amount
0,2023-01-01,blouses,long-sleeve blouses,96
1,2023-01-02,blouses,dresses,52
2,2023-01-03,tops,short-sleeve blouses,78
3,2023-01-04,blouses,dresses,93
4,2023-01-05,dresses,t-shirts,98
...,...,...,...,...
9995,2050-05-14,blouses,short-sleeve blouses,43
9996,2050-05-15,tops,short-sleeve blouses,48
9997,2050-05-16,blouses,t-shirts,63
9998,2050-05-17,blouses,tunics,22


### Code Extractor

In [196]:
## Rewrite code
def code_extractor(generated_code:str) -> str:
    code_extractor_prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are a Code Extractor Agent. \n
        You take in codes that might have some texts added to it. \n

        Only extract the code and return it as the only output. \n

        Return only the code and nothing added to it.
        <|eot_id|><|start_header_id|>user<|end_header_id|>
        CODE: {generated_code} \n\n

        <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["generated_code"],
    )

    code_extractor_chain = code_extractor_prompt | GROQ_LLM | StrOutputParser()
    output = code_extractor_chain.invoke({"generated_code":generated_code})
    return output

In [110]:
# extracted_code = code_extractor_chain.invoke({"generated_code": rewritten_code})

In [112]:
# print(extracted_code)

```
import pandas as pd
import numpy as np
import datetime as dt

column_names = ['date', 'product_category', 'product_sub_category', 'sales_amount']
data_types = {'date': 'datetime64[D]', 'product_category': 'category', 'product_sub_category': 'category', 'sales_amount': 'float64'}

data_size = 10000

data = {
    'date': [dt.datetime(2023, 1, 1) + dt.timedelta(days=i) for i in range(data_size)],
    'product_category': np.random.choice(['blouses', 'tops', 'dresses'], size=data_size),
    'product_sub_category': np.random.choice(['long-sleeve blouses', 'short-sleeve blouses', 'tunics', 't-shirts', 'dresses'], size=data_size),
    'sales_amount': np.random.randint(1, 100, size=data_size)
}

df = pd.DataFrame(data)

print(df)


### Code Modifier

In [135]:
## Rewrite code
def code_modifier(generated_code: str) -> str:
  code_modifier_prompt = PromptTemplate(
      template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
      You are a Coding Agent. You receive from CODES texts that contain a full code that creates a synthetic data for a client. \n
      Your job is to convert this code into a high level Class that has a crate function whereby when called the data is created.
      You are returning a high level class that has a create function that creates the data.
      This is how you should structure the class:
        import the necessary libraries
        classname:   SyntheticDataGenerator
          method:      create - creates the synthetic data
          method: save_to_csv - saves the data to a csv file
          method: generate_and_save_data - generates and saves the data to a csv file

      Return just the code, no extra text.
      <|eot_id|><|start_header_id|>user<|end_header_id|>
      CODEs: {generated_code} \n\n

      <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
      input_variables=["generated_code"],
  )

  code_modifier_chain = code_modifier_prompt | GROQ_LLM | StrOutputParser()
  output = code_modifier_chain.invoke({"generated_code":generated_code})
  return output

In [134]:
modified_code = code_modifier_chain.invoke({"generated_code": extracted_code})

In [136]:
print(modified_code)

```
import pandas as pd
import numpy as np
import datetime as dt

class SyntheticDataGenerator:
    def __init__(self, data_size=10000):
        self.data_size = data_size

    def create(self):
        column_names = ['date', 'product_category', 'product_sub_category', 'sales_amount']
        data_types = {'date': 'datetime64[D]', 'product_category': 'category', 'product_sub_category': 'category', 'sales_amount': 'float64'}

        data = {
            'date': [dt.datetime(2023, 1, 1) + dt.timedelta(days=i) for i in range(self.data_size)],
            'product_category': np.random.choice(['blouses', 'tops', 'dresses'], size=self.data_size),
            'product_sub_category': np.random.choice(['long-sleeve blouses', 'short-sleeve blouses', 'tunics', 't-shirts', 'dresses'], size=self.data_size),
            'sales_amount': np.random.randint(1, 100, size=self.data_size)
        }

        self.df = pd.DataFrame(data)

    def save_to_csv(self, filename):
        self.df.to_csv(filename

In [137]:
# code cleaner
def remove_triple_quotes(text):
    if (text.startswith("'''") and text.endswith("'''")) or (text.startswith("```") and text.endswith("```")):
        return text[3:-3]  # Remove triple quotes from both ends
    elif text.startswith("'''") or text.startswith("```"):
        return text[3:]  # Remove triple quotes from the beginning
    elif text.endswith("'''") or text.endswith("```"):
        return text[:-3]  # Remove triple quotes from the end
    else:
        return text

cleaned_code = remove_triple_quotes(modified_code)

In [139]:
print(cleaned_code)


import pandas as pd
import numpy as np
import datetime as dt

class SyntheticDataGenerator:
    def __init__(self, data_size=10000):
        self.data_size = data_size

    def create(self):
        column_names = ['date', 'product_category', 'product_sub_category', 'sales_amount']
        data_types = {'date': 'datetime64[D]', 'product_category': 'category', 'product_sub_category': 'category', 'sales_amount': 'float64'}

        data = {
            'date': [dt.datetime(2023, 1, 1) + dt.timedelta(days=i) for i in range(self.data_size)],
            'product_category': np.random.choice(['blouses', 'tops', 'dresses'], size=self.data_size),
            'product_sub_category': np.random.choice(['long-sleeve blouses', 'short-sleeve blouses', 'tunics', 't-shirts', 'dresses'], size=self.data_size),
            'sales_amount': np.random.randint(1, 100, size=self.data_size)
        }

        self.df = pd.DataFrame(data)

    def save_to_csv(self, filename):
        self.df.to_csv(filename, i

In [146]:
# Code saver
def code_saver(code, filepath):
    with open(filepath, 'w') as f:
        f.write(code)

In [147]:
code_saver(cleaned_code, "synthetic_data_generator.py")

In [168]:
# code_excecutor
import json
import traceback

file_path = 'synthetic_data_generator.py'
def code_executor():
    # Check if the file exists
    if os.path.exists(file_path):
        try:
            # importing the class from the file
            from synthetic_data_generator import SyntheticDataGenerator

            # creating an instance of the class
            data = SyntheticDataGenerator()

            # calling the create function
            data.generate_and_save_data('data.csv')

            return "Data generated successfully!"
        except Exception as e:
        # Capture the complete error, including the traceback
            error = {
                "error": str(e),
                "traceback": traceback.format_exc()
            }
                # saving the error in a json file
            with open('error.json', 'w') as f:
                json.dump(error, f)
            
            return 'Error'


In [169]:
code_executor()

'Data generated successfully!'

### Merging all together

In [191]:
CHAT_HISTORY =  {
    "messages": [
        {"user": "USER", "text": "Hi, I'm trying to create a dataset for sales in a city. I'm looking to analyze the sales of fashion products in New York City. I want to track the sales of specific products to understand their market performance.", "timestamp": "2023-02-10T14:25:00"},
        {"ai": "AI", "text": "Sounds like an interesting project! Can you please tell me more about what you're trying to achieve? What specific problem are you trying to solve?", "timestamp": "2023-02-10T14:26:00"},
        {"user": "USER", "text": "The problem I'm trying to solve is to analyze the sales trends of blouses, tops, and dresses in New York City. I want to understand which products are most popular and identify any seasonal patterns.", "timestamp": "2023-02-10T14:27:00"},
        {"ai": "AI", "text": "Got it! So, you're looking to analyze sales data for blouses, tops, and dresses in New York City. Can you please list the columns you'd like to include in the dataset?", "timestamp": "2023-02-10T14:28:00"},
        {"user": "USER", "text": "I'd like to include columns for date, product category (blouses, tops, dresses), product sub-category (e.g. long-sleeve blouses), and sales amount.", "timestamp": "2023-02-10T14:30:00"},
        {"ai": "AI", "text": "That makes sense! What data types do you expect for each column? For example, would the sales amount be a numerical value?", "timestamp": "2023-02-10T14:31:00"},
        {"user": "USER", "text": "Yes, the sales amount would be a numerical value. And the date would be a date type. The product categories would be categorical.", "timestamp": "2023-02-10T14:33:00"},
        {"ai": "AI", "text": "Got it! For the sales amount, I assume you want a numerical value, such as a float or integer. For the date, I assume it would be a date data type. And for the product categories, I assume it would be a categorical value, such as strings or categorical data types like enum or categorical.", "timestamp": "2023-02-10T14:34:00"},
        {"user": "USER", "text": "Yes, that's correct! I'd also like to know how many rows of data you'd like to generate.", "timestamp": "2023-02-10T14:35:00"},
        {"ai": "AI", "text": "I can generate a dataset with a specified number of rows. How many rows would you like the dataset to have?", "timestamp": "2023-02-10T14:36:00"},
        {"user": "USER", "text": "I'd like the dataset to have 10,000 rows.", "timestamp": "2023-02-10T14:37:00"},
        {"ai": "AI", "text": "Okay, I'll generate a dataset with 10,000 rows. Would you like to specify any variability in the data?", "timestamp": "2023-02-10T14:38:00"},
        {"user": "USER", "text": "No, I don't need any variability in the data.", "timestamp": "2023-02-10T14:39:00"},
        {"ai": "AI", "text": "Alright, I've generated the dataset with the specified columns, data types, and number of rows. I've excluded variability to ensure consistent data. ALL REQUIREMENTS RECORDED", "timestamp": "2023-02-10T14:40:00"}
    ]
}


In [194]:
print(initiator(CHAT_HISTORY))

START


In [197]:
def Data_Generator(CHAT_HISTORY: list) -> str:
    # Initiating the flow
    initiation_ = initiator(CHAT_HISTORY)
    print(initiation_)
    if initiation_ == "START":
        # Now proceeding with the chat flow
        try:
            # chat_reviewer section to review chats
            chat_review_ = chat_reviewer(CHAT_HISTORY)
            # Project Requirement Extractor
            extracted_req = requirement_extractor(CHAT_HISTORY, chat_review_)
            # Python Code Generator
            code_output = code_generator(CHAT_HISTORY, chat_review_, extracted_req)
            # Extracting only the code
            extracted_code = code_extractor(code_output)
            # Modifying the code into classes
            modified_code = code_modifier(extracted_code)
            # Removing backstrings from the code
            cleaned_code = remove_triple_quotes(modified_code)
            # saving the cleaned code
            code_saver(cleaned_code, "synthetic_data_generator.py")
            # Exxecuting the codes
            execute = code_executor()
            return execute
        except Exception as e:
        # Capture the complete error, including the traceback
            error = {
                "error": str(e),
                "traceback": traceback.format_exc()
            }
                # saving the error in a json file
            with open('error.json', 'w') as f:
                json.dump(error, f)
    

In [198]:
print(Data_Generator(CHAT_HISTORY))

START
Data generated successfully!


In [189]:
from faker import Faker

# Create an instance of Faker
fake = Faker()

# Generate a fake name
print("Fake Name:", fake.name())

# Generate a fake address
print("Fake Address:", fake.address())

# Generate a fake email
print("Fake Email:", fake.email())

# Generate a fake date of birth within a specific age range (18 to 90 years old)
print("Fake Date of Birth:", fake.date_of_birth(minimum_age=18, maximum_age=90))

# Generate a random date
print("Random Date:", fake.date())

# Generate a random number with 5 digits
print("Random Number (5 digits):", fake.random_number(digits=5))

# Generate a fake phone number
print("Fake Phone Number:", fake.phone_number())

# Generate a fake company name
print("Fake Company Name:", fake.company())

# Generate a block of text with a maximum of 200 characters
print("Fake Text (max 200 characters):", fake.text(max_nb_chars=200))

Fake Name: Sara Hall
Fake Address: 19553 Dana Knoll Suite 321
Port Rodney, AK 07353
Fake Email: dyoung@example.org
Fake Date of Birth: 1969-07-21
Random Date: 1981-07-23
Random Number (5 digits): 71356
Fake Phone Number: +1-534-673-2815x0712
Fake Company Name: Smith, Lee and Lucas
Fake Text (max 200 characters): Buy he rest language hard realize soon. Sound act boy player carry season. Actually just office even.
Guy listen bar think resource. Defense line less turn season low. Deal seat success.


In [172]:
# Generate a fake date within a specific range (e.g., between 2020-01-01 and 2021-01-01)
from datetime import datetime
start_date = datetime(2020, 1, 1)
end_date = datetime(2021, 1, 1)
fake_date_within_range = fake.date_time_between_dates(datetime_start=start_date, datetime_end=end_date)
print("Fake Date within Range (2020-01-01 to 2021-01-01):", fake_date_within_range)

Fake Date within Range (2020-01-01 to 2021-01-01): 2020-08-27 02:28:10


'Data has been saved successfully to synthetic_data.py.'

In [43]:
import pandas as pd
from faker import Faker
from datetime import date, timedelta

# Create a Faker instance
fake = Faker()

# Set the number of rows
data_size = 10000

# Create a list to store the data
data = []

# Iterate through the number of rows
for i in range(data_size):
    # Generate the date
    date_obj = date.fromtimestamp(int(fake.random_int(min=1643728000, max=1646324800)))
    date_str = date_obj.strftime('%Y-%m-%d')

    # Generate the product category (blouses, tops, dresses)
    product_categories = ['blouses', 'tops', 'dresses']
    product_category = fake.random_element(elements=product_categories)

    # Generate the product sub-category (e.g. long-sleeve blouses)
    product_sub_categories = ['long-sleeve blouses', 'short-sleeve blouses', 'tank tops']
    product_sub_category = fake.random_element(elements=product_sub_categories)

    # Generate the sales amount
    sales_amount = fake.random_int(min=10, max=1000)

    # Append the data to the list
    data.append({
        'date': date_str,
        'product_category': product_category,
        'product_sub_category': product_sub_category,
        'sales_amount': sales_amount
    })

# Convert the list to a Pandas DataFrame
df = pd.DataFrame(data)

In [44]:
df

,date,product_category,product_sub_category,sales_amount
0,2022-02-05,dresses,short-sleeve blouses,812
1,2022-02-11,dresses,long-sleeve blouses,133
2,2022-02-11,blouses,short-sleeve blouses,833
3,2022-03-03,dresses,tank tops,737
4,2022-02-17,dresses,tank tops,116
...,...,...,...,...
9995,2022-03-02,dresses,tank tops,304
9996,2022-02-25,tops,tank tops,75
9997,2022-02-18,dresses,tank tops,245
9998,2022-02-22,dresses,long-sleeve blouses,626
